In [ ]:
from pyspark.sql import SparkSession
import pandas
from pyspark.sql.types     import *
from pyspark.sql.functions import *
from datetime              import datetime
from pyspark.sql.types     import IntegerType

In [ ]:
appNameSpark = "Carrega dados covid"
hdfsNodeRaw = "hdfs://hadoop-namenode:9000/gft/raw/covid"
hdfsNodeTrusted = "hdfs://hadoop-namenode:9000/gft/trusted/covid"
hdfsNodeRefined = "hdfs://hadoop-namenode:9000/gft/refined/covid"
header =True
delimiter = ','

In [ ]:
spark = SparkSession.builder \
                    .master("local[1]")\
                    .appName(appNameSpark) \
                    .getOrCreate()

###Carrega arquivos csv da raw

In [ ]:
df_csv = spark.read\
                .option('header', header)\
                .option('delimiter', delimiter)\
                .option("escape", "\"")\
                .csv(str(hdfsNodeRaw))

In [ ]:
df_csv.show(10)

###Carrega arquivos parquet da trusted

In [ ]:
df_parquet = spark.read\
                .option('header', header)\
                .option('delimiter', delimiter)\
                .option("escape", "\"")\
                .parquet(str(hdfsNodeTrusted))

In [ ]:
df_parquet.limit(10).toPandas().head()

In [ ]:
df_covid =  df_parquet.withColumn('ID', col('_id').cast(StringType()))\
                    .withColumn('DT_NOTIFICACAO', col('dataNotificacao').cast(StringType()))\
                    .withColumn('CNES', col('cnes').cast(StringType()))\
                    .withColumn('OC_SUSPEITA_CLI', col('ocupacaoSuspeitoCli').cast(IntegerType()))\
                    .withColumn('OC_SUSPEITA_UTI', col('ocupacaoSuspeitoUti').cast(IntegerType()))\
                    .withColumn('OC_CONFIRMADA_CLI', col('ocupacaoConfirmadoCli').cast(IntegerType()))\
                    .withColumn('OC_CONFIRAMDA_UTI', col('ocupacaoConfirmadoUti').cast(IntegerType()))\
                    .withColumn('OC_COVID_CLI', col('ocupacaoCovidCli').cast(IntegerType()))\
                    .withColumn('OC_COVID_UTI', col('ocupacaoCovidUti').cast(IntegerType()))\
                    .withColumn('OC_HOSPITALAR_CLI', col('ocupacaoHospitalarCli').cast(IntegerType()))\
                    .withColumn('OC_HOSPITALAR_UTI', col('ocupacaoHospitalarUti').cast(IntegerType()))\
                    .withColumn('SD_SUSPEITA_OBITOS', col('saidaSuspeitaObitos').cast(IntegerType()))\
                    .withColumn('SD_SUSPEITA_ALTAS', col('saidaSuspeitaAltas').cast(IntegerType()))\
                    .withColumn('SD_CONFIRMADA_OBITOS', col('saidaConfirmadaObitos').cast(IntegerType()))\
                    .withColumn('SD_CONFIRMADA_ALTAS', col('saidaConfirmadaAltas').cast(IntegerType()))\
                    .withColumn('ORIGEM', col('origem').cast(StringType()))\
                    .withColumn('CD_USUARIO', col('_p_usuario').cast(StringType()))\
                    .withColumn('UF', col('estado').cast(StringType()))\
                    .withColumn('MUNICIPIO', col('municipio').cast(StringType()))

In [ ]:
df_covid.write\
                        .mode('append')\
                        .parquet(str(hdfsNodeRefined))